# Chapter 5 Automatic Merge Search

In this course, we will explore an advanced retrieval aggregation (RAG) technique - automatic merge retrieval. This approach solves a key problem encountered when using traditional RAG: when retrieving information chunks to fill the language model (LLM) context, if these chunks are too fragmented, it will affect the coherence of the context, especially when the size of these chunks is small. To overcome this challenge, we will adopt an automatic merge heuristic method, which ensures the coherence of the context by merging smaller information chunks into larger parent chunks. Next, we will introduce how to implement this technique in detail and demonstrate its application through practical examples.

First, we need to prepare the environment and import the necessary libraries. Please note that in order to avoid unnecessary warning messages interfering with our operations, we will use the warnings library to ignore them.

In [1]:
import warnings
warnings.filterwarnings('ignore')

Next, we’ll import some utility functions, set up our OpenAI API key, and prepare our documentation, which will serve as our search source.

In [2]:
import utils

import os
import openai
# Get the OpenAI API key through utils and set it
openai.api_key = utils.get_openai_api_key()

# Or fill in your OpenAI API key here
# openai.api_key = ""

# Or customize the API key and API base address, which can be used for third-party API services
# openai.api_key = "sk- "
# openai.api_base = " "

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


We will use the llama_index library to read and process documents. In this example, we will load a PDF file, but you are encouraged to try it with your own documents.

In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./data/人工智能.pdf"]
).load_data()

In [4]:
documents_en = SimpleDirectoryReader(
    input_files=["./data/eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

To verify that our documents have been loaded correctly, let's print out the type of document, the number of documents, and the contents of the first document.

In [5]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

7 

<class 'llama_index.schema.Document'>
Doc ID: 35bf3256-2c52-4995-ae3b-311956872365
Text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 2/13“⼈⼯智能”的各地常⽤名称 中国⼤陆⼈⼯智能 台湾⼈⼯智慧
港澳⼈⼯智能 新⻢⼈⼯智能、⼈⼯智慧 ⽇韩⼈⼯知能 越南智慧⼈造 [展开] [展开] [展开] [展开] [展开] [展开]⼈⼯智能系列内容
主要⽬标 实现⽅式 ⼈⼯智能哲学 历史 技术 术语⼈⼯智能（英语：artiﬁcial intelligence ，缩写为
AI）亦称机器智能，指由⼈制造出来的机器所表现出来的智能。通常⼈⼯
智能是指⽤普通计算机程序来呈现⼈类智能的技术。该词也指出研究这样的智能系统是否能够实现，以及如何实现。同 时，通过 医学 、神经科学
、机器⼈学 及...


In [6]:
print(type(documents_en), "\n")
print(len(documents_en), "\n")
print(type(documents_en[0]))
print(documents_en[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: ab54b5ce-57c8-4187-91c4-de8c3d44cf4e
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## 1. Automatic merge search settings

We merge all documents into one large document object for automatic merge retrieval.

In [7]:
from llama_index import Document

# Merge all document text into one large document
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
document_en = Document(text="\n\n".join([doc.text for doc in documents_en]))

In [9]:
# Replace Chinese punctuation marks with English punctuation marks for easy subsequent processing
# If it is an English document, you can skip this step
# If not processed, it will lead to the inability to correctly segment Chinese sentences, which will affect the size of the subsequent sentence_window and cause the input length to exceed the maximum limit of gpt-3.5-turbo
document.text=document.text.replace('。','. ')
document.text=document.text.replace('！','! ')
document.text=document.text.replace('？','? ')

Now, we will create a hierarchy node parser. This parser will help us create a hierarchy based on the document content to facilitate the subsequent automatic merging operation.

In [10]:
from llama_index.node_parser import HierarchicalNodeParser

# Create a hierarchy node parser with default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

Through the node parser, we get all the nodes in the document.

In [11]:
# Get a node from the document
nodes = node_parser.get_nodes_from_documents([document])

In [12]:
nodes_en = node_parser.get_nodes_from_documents([document_en])

To show how this is done, we will take out some leaf nodes and look at their contents. Leaf nodes are the smallest blocks in the hierarchy and they will directly participate in the subsequent merge process.

In [13]:
from llama_index.node_parser import get_leaf_nodes

# Get the leaf nodes and print the text of one of them as an example
leaf_nodes = get_leaf_nodes(nodes)
print(leaf_nodes[30].text)

⼈⼯智能⼯具
与司法部门的⼈类法官相辅相成，提供客观、⼀致的风险评估[13].


In [14]:
leaf_nodes_en = get_leaf_nodes(nodes_en)
print(leaf_nodes_en[30].text)

But this became less important as numerical linear algebra libraries matured.
Deep learning is still an emerging technology, so when you train a neural network and the 
optimization algorithm struggles to converge, understanding the math behind gradient 
descent, momentum, and the Adam  optimization algorithm will help you make better decisions. 
Similarly, if your neural network does something funny — say, it makes bad predictions on 
images of a certain resolution, but not others — understanding the math behind neural network 
architectures puts you in a better position to figure out what to do.
Of course, I also encourage learning driven by curiosity.


We can also view the parent nodes of these leaf nodes. The parent node contains the contents of all its child nodes, which is very important for us to understand the automatic merging process.

In [15]:
# Get the node by ID and print the parent node text of a leaf node as an example
nodes_by_id = {node.node_id: node for node in nodes}

parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
print(parent_node.text)

⼈⼯智能为地⽅政府服务带来技术突破. ⼈⼯智能代理协助城市规划者基于⽬标导向的蒙特卡罗树搜索进⾏场景规划. ⽬标推理⼈⼯智能代理提供最佳的⼟
地利⽤解决⽅案，帮助我们制定民主的城市⼟地利⽤规划. ⼈⼯智能利⽤在线数据来监控和修改环境威胁政策. 在2019 年⽔危机期间，潜在狄利克雷分配⽅
法确定了Twitter (X) 中讨论最多的主题，这是⼀种朴素的推⽂分类⽅法，对⼲旱的影响和原因、政府响应和潜在解决⽅案等主题进⾏了分类. ⼈⼯智能⼯具
与司法部门的⼈类法官相辅相成，提供客观、⼀致的风险评估[13].


In [16]:
nodes_by_id_en = {node.node_id: node for node in nodes_en}

parent_node_en = nodes_by_id_en[leaf_nodes_en[30].parent_node.node_id]
print(parent_node_en.text)

PAGE 12Should You 
Learn Math to 
Get a Job in AI? CHAPTER 3
LEARNING

PAGE 13Should you Learn Math to Get a Job in AI? CHAPTER 3
Is math a foundational skill for AI? It’s always nice to know more math! But there’s so much to 
learn that, realistically, it’s necessary to prioritize. Here’s how you might go about strengthening 
your math background.
To figure out what’s important to know, I find it useful to ask what you need to know to make 
the decisions required for the work you want to do. At DeepLearning.AI, we frequently ask, 
“What does someone need to know to accomplish their goals?” The goal might be building a 
machine learning model, architecting a system, or passing a job interview.
Understanding the math behind algorithms you use is often helpful, since it enables you to 
debug them. But the depth of knowledge that’s useful changes over time. As machine learning 
techniques mature and become more reliable and turnkey, they require less debugging, and a 
shallower understand

### 1.1. Build index

We will build an index and define a retriever in order to run the query engine and evaluate the performance of automatic merge retrieval.

Continuing to build our automatic merge retrieval system, we need to build an index using OpenAI's language model and some custom components that will support our automatic merge logic.

First, we will initialize an OpenAI language model, here we choose to use GPT-3.5 Turbo, which is a powerful model suitable for a variety of NLP tasks.

In [17]:
from llama_index.llms import OpenAI

# Using OpenAI's GPT-3.5 Turbo model
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

Next, we will create a service context object. This object will contain all the services we need, such as embedding models and node parsers, so that our system can correctly perform automatic merge retrieval.

In [18]:
from llama_index import ServiceContext

# Create a service context, including LLM model and node parser
auto_merging_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    node_parser=node_parser,
)

In [19]:
# English embeding model
auto_merging_context_en = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

Now, we need to store our document nodes in an index. This index will support fast retrieval operations and allow us to dynamically merge document nodes as needed.

In [20]:
from llama_index import VectorStoreIndex, StorageContext

# Create a storage context and add a document node
storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

# Create automatic merge index
automerging_index = VectorStoreIndex(
    leaf_nodes, storage_context=storage_context, service_context=auto_merging_context
)

# Persist the index for future use
automerging_index.storage_context.persist(persist_dir="./merging_index")

In [21]:
# Create a storage context and add a document node
storage_context_en = StorageContext.from_defaults()
storage_context_en.docstore.add_documents(nodes_en)

# Create automatic merge index
automerging_index_en = VectorStoreIndex(
    leaf_nodes_en, storage_context=storage_context_en, service_context=auto_merging_context_en
)

# Persist the index for future use
automerging_index_en.storage_context.persist(persist_dir="./merging_index_en")

To ensure that our system is able to reload the index when needed, we also provide an optional code block that checks if the index file exists and loads or rebuilds it as needed.

In [22]:
# Optional code block: Check if the index file exists and rebuild it if it does not exist
import os
from llama_index import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index import load_index_from_storage

if not os.path.exists("./merging_index"):
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    automerging_index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            service_context=auto_merging_context
        )

    automerging_index.storage_context.persist(persist_dir="./merging_index")
else:
    automerging_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index"),
        service_context=auto_merging_context
    )


In [23]:
if not os.path.exists("./merging_index_en"):
    storage_context_en = StorageContext.from_defaults()
    storage_context_en.docstore.add_documents(nodes_en)

    automerging_index_en = VectorStoreIndex(
            leaf_nodes_en,
            storage_context=storage_context_en,
            service_context=auto_merging_context_en
        )

    automerging_index_en.storage_context_en.persist(persist_dir="./merging_index_en")
else:
    automerging_index_en = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./merging_index_en"),
        service_context=auto_merging_context_en
    )

### 1.2. Define the retriever and run the query engine

Next, we will define the retriever and run the query engine. During this process, we will set up an automatic merge retriever, which will control the merging logic of document nodes to ensure that the retrieved context is coherent.

In [24]:
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.retrievers import AutoMergingRetriever
from llama_index.query_engine import RetrieverQueryEngine

# Convert an auto-merge index to a retriever
automerging_retriever = automerging_index.as_retriever(
    similarity_top_k=12
)

# Define automatic merge retriever
retriever = AutoMergingRetriever(
    automerging_retriever, 
    automerging_index.storage_context, 
    verbose=True
)

# Define the re-ranking module
rerank = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

# Create an automatic merge search query engine
auto_merging_engine = RetrieverQueryEngine.from_args(
    automerging_retriever, node_postprocessors=[rerank]
)

In [25]:
# Convert an auto-merge index to a retriever
automerging_retriever_en = automerging_index_en.as_retriever(
    similarity_top_k=12
)

# Define automatic merge retriever
retriever_en = AutoMergingRetriever(
    automerging_retriever_en, 
    automerging_index_en.storage_context, 
    verbose=True
)

# Define the re-ranking module
rerank_en = SentenceTransformerRerank(top_n=6, model="BAAI/bge-reranker-base")

# Create an automatic merge search query engine
auto_merging_engine_en = RetrieverQueryEngine.from_args(
    automerging_retriever_en, node_postprocessors=[rerank_en]
)

To test our automatic merge retrieval system, we will use an example query: “What are the threats of AI to humanity?” and show the retrieved responses.

In [26]:
# Execute a query using the query engine
auto_merging_response = auto_merging_engine.query(
    "AI对人类的威胁有哪些？"
)

In [27]:
# Execute a query using the query engine
auto_merging_response_en = auto_merging_engine_en.query(
    "What is the importance of networking in AI?"
)

We can use the provided utility functions to display the retrieved response beautifully.

In [28]:
from llama_index.response.notebook_utils import display_response

# Display query responses
display_response(auto_merging_response)

**`Final Response:`** AI对人类的威胁包括：AI会遵循科技发展的加速度理论、AI可能会有自我改造创新的能力、AI进步的速度远远超过人类、人类会有被灭绝的危机。

In [29]:
display_response(auto_merging_response_en)

**`Final Response:`** Networking in AI is crucial as it allows individuals to build a strong professional community that can provide valuable information, support, and opportunities. By connecting with others in the field, individuals can receive guidance, advice, and potential referrals to employers. Additionally, networking helps individuals stay updated on the latest trends and developments in AI, fostering continuous learning and growth within the community.

## 2. Integrate all steps

We will integrate all the above steps and evaluate our automerging retrieval system through TruLens to verify its performance.

To integrate the various parts of automerging retrieval together and evaluate it, we will define two high-level functions: `build_automerging_index` and `get_automerging_query_engine`. These functions will help us build the index and initialize the query engine, allowing us to test and evaluate different configurations in a more concise way.

First, we define the `build_automerging_index` function, which is responsible for building the automerging index. This function will accept documents, language models, embedding models, and save directories as parameters and build an index containing all document nodes.

Next, we define the `get_automerging_query_engine` function, which is responsible for creating a query engine from the automerging index. It sets up the retriever to retrieve based on the top K values ​​of similarity and applies reranking logic to optimize the retrieval results.

In [30]:
import os

from llama_index import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
from llama_index.node_parser import HierarchicalNodeParser
from llama_index.node_parser import get_leaf_nodes
from llama_index import StorageContext, load_index_from_storage
from llama_index.retrievers import AutoMergingRetriever
from llama_index.indices.postprocessor import SentenceTransformerRerank
from llama_index.query_engine import RetrieverQueryEngine


# Define a function to build an automatic merge index
def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


# Define the function to get the automatic merge query engine
def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

Using these two functions, we can easily build indexes and query engines, and test the effect of automatic merge retrieval through different parameter configurations. The following is an example of how to use these functions:

In [31]:
from llama_index.llms import OpenAI

# Build automatic merge index
index = build_automerging_index(
    [document],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index",
)


In [32]:
index_en = build_automerging_index(
    [document_en],
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    save_dir="./merging_index_en",
)

In [33]:
# Get the automatic merge query engine
query_engine = get_automerging_query_engine(index, similarity_top_k=6)

In [34]:
query_engine_en = get_automerging_query_engine(index_en, similarity_top_k=6)

## 3. TruLens Evaluation

After building the auto-merge index and query engine, we will use TruLens to evaluate the performance of the auto-merge retriever. TruLens is a tool for evaluating AI models that provides detailed performance metrics. We will first reset the TruLens database and then perform the evaluation.

In [35]:
from trulens_eval import Tru

# Reset the TruLens database
Tru().reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


We will evaluate the performance of two-layer and three-layer auto-merge structures respectively, and use TruLens to record and present the evaluation results. This will help us understand the impact of different hierarchy configurations on performance and provide us with insights to optimize the auto-merge retrieval strategy.

Before doing the TruLens evaluation, we need to set up the auto-merge index and query engine for two different hierarchies. The two structures are two-layer and three-layer. By comparing the performance of these two structures, we can better understand the impact of hierarchy depth on the auto-merge retrieval effect.

### 3.1. Two-layer index construction

First, we will build an auto-merge index with only two levels. This structure simplifies the process of creating the index and reduces the amount of work required in the retrieval step.

In [36]:
# Build an automatic merge index with a two-layer structure
auto_merging_index_0 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index_0",
    chunk_sizes=[2048,512],
)

In [37]:
auto_merging_index_0_en = build_automerging_index(
    documents_en,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_0_en",
    chunk_sizes=[2048,512],
)

Next, we set up the query engine for this two-level auto-merge index, using the same top K and re-ranking steps as before.

In [38]:
# Get the automatic merge query engine for the two-layer structure
auto_merging_engine_0 = get_automerging_query_engine(
    auto_merging_index_0,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [39]:
auto_merging_engine_0_en = get_automerging_query_engine(
    auto_merging_index_0_en,
    similarity_top_k=12,
    rerank_top_n=6,
)

In order to perform a TruLens evaluation, we need to define a TruLens recorder, which will record the query engine's responses and the evaluation results.

In [ ]:
from utils import get_prebuilt_trulens_recorder

# Prepare for evaluation using the TruLens recorder
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_0,
    app_id ='app_0'  # 设置应用ID
)

In [40]:
# Prepare for evaluation using the TruLens recorder
tru_recorder_en = get_prebuilt_trulens_recorder(
    auto_merging_engine_0_en,
    app_id ='app_0_en'  # 设置应用ID
)

We then load the evaluation questions from the pre-prepared question file and define a function to run the evaluation. This function will iterate over all the evaluation questions and record the query results for each question.

In [43]:
# Load the evaluation questions
eval_questions = []
with open('./data/generated_questions.txt', 'r', encoding="utf8") as file:
    for line in file:
        item = line.strip()  # 移除换行符
        eval_questions.append(item)

In [43]:
# Load the evaluation questions
eval_questions_en = []
with open('./data/generated_questions_en.txt', 'r') as file:
    for line in file:
        item = line.strip()  # 移除换行符
        eval_questions_en.append(item)

Finally, we run the evaluation using the function defined above and retrieve and display the leaderboard via TruLens.

In [41]:
import time
# Define the function that runs the evaluation
def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            response = query_engine.query(question)
            time.sleep(2)
# Run the evaluation here and record the results

In [47]:
# Evaluate the two-layer structure
run_evals(eval_questions, tru_recorder, auto_merging_engine_0)

> Merging 3 nodes into parent node.
> Parent node id: d33cd5ff-82dc-4ba8-a6d7-c3b77f35315b.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 3/13
本体论将知识表示为⼀个领域内的⼀
组概...

> Merging 4 nodes into parent node.
> Parent node id: 4a656a65-c52b-410b-8152-1545952c47ad.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 6/13智能是否可以使⽤⾼级符号表达，如词和想法...

> Merging 3 nodes into parent node.
> Parent node id: cc7c42fb-22be-4ba5-b8c4-281f4b8b6236.
> Parent node text: 2/2/24, 2:43 PM ⼈⼯智能  - 维基百科，⾃由的百科全书
https://zh.wikipedia.org/wiki/ ⼈⼯智能 4/13
Kismet，⼀个具有表情等社交能⼒
...

> Merging 3 nodes into parent node.
> Parent node id: fded6622-1120-4fae-8784-fe5052059204.
> Parent node text: 2017年6⽉份马云在美国底特律举⾏“链接世界”（Gateway 17）产业⼤会，会上提出⼈⼯智能可能导致第三次世界⼤战，因为前两次产业⾰命都导致两次⼤
战，战争原因并⾮这些创新发明本⾝，⽽是发...

> Merging 2 nodes into parent node.
> Parent node id: d6a24814-902b-4670-9bad-70f7a5be9f03.
> Parent node text: The Behavioral and Brain Science

In [44]:
# Evaluate the two-layer structure
run_evals(eval_questions_en, tru_recorder_en, auto_merging_engine_0_en)

> Merging 1 nodes into parent node.
> Parent node id: 95c8fa62-58cc-46a9-b08c-77fc05473471.
> Parent node text: PAGE 26If you’re considering a role switch, a startup can be an easier place to do it than a big ...

> Merging 1 nodes into parent node.
> Parent node id: db5ac140-f84a-4b8f-89c3-e4e47887874b.
> Parent node text: PAGE 25Finding a job has a few predictable steps that include selecting the companies to which yo...

> Merging 1 nodes into parent node.
> Parent node id: 9145b623-e40f-4038-95af-d99ee4d6cba6.
> Parent node text: PAGE 33Choose who to work with. It’s tempting to take a position because of the projects you’ll w...

> Merging 1 nodes into parent node.
> Parent node id: 34343c7a-bbc6-4d62-8dca-cea4fab0cdf0.
> Parent node text: PAGE 23Each project is only one step on a longer journey, hopefully one that has a positive impac...

> Merging 1 nodes into parent node.
> Parent node id: 6abfebf1-807f-4961-925c-4506de715664.
> Parent node text: PAGE 29If you’re preparing to sw

In [45]:
from trulens_eval import Tru

# Get and display the leaderboard
Tru().get_leaderboard(app_ids=[])

,Answer Relevance,Groundedness,Context Relevance,latency,total_cost
app_id,,,,,
app_0_en,0.979167,0.746528,0.413333,6.5,0.00375


In [29]:
# Run the TruLens dashboard to view detailed evaluation results
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.2.7:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 3.2. Three-layer index construction

Next, we will try a more complex automatic merge index construction method, which contains three layers and may improve the quality of retrieval.

In [47]:
# Build an automatic merge index with a three-layer structure
auto_merging_index_1 = build_automerging_index(
    documents,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-zh-v1.5",
    save_dir="merging_index_1",
    chunk_sizes=[2048, 512, 128],  # 三层结构的块大小设置
)

In [48]:
# Build an automatic merge index with a three-layer structure
auto_merging_index_1_en = build_automerging_index(
    documents_en,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.1),
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index_1_en",
    chunk_sizes=[2048, 512, 128],  # 三层结构的块大小设置
)

Set up the query engine for this three-tiered automatic merge index.

In [49]:
# Get the three-layer automatic merge query engine
auto_merging_engine_1 = get_automerging_query_engine(
    auto_merging_index_1,
    similarity_top_k=12,
    rerank_top_n=6,
)

In [50]:
# Get the three-layer automatic merge query engine
auto_merging_engine_1_en = get_automerging_query_engine(
    auto_merging_index_1_en,
    similarity_top_k=12,
    rerank_top_n=6,
)

Similarly, we define a TruLens recorder and run the evaluation with the same evaluation questions.

In [32]:
# Prepare TruLens recorder for three-layer structure
tru_recorder = get_prebuilt_trulens_recorder(
    auto_merging_engine_1,
    app_id ='app_1'  # 设置应用ID
)

In [ ]:
# Prepare TruLens recorder for three-layer structure
tru_recorder_en = get_prebuilt_trulens_recorder(
    auto_merging_engine_1_en,
    app_id ='app_1_en'  # 设置应用ID
)

In [33]:
# Evaluate the three-layer structure
run_evals(eval_questions, tru_recorder, auto_merging_engine_1)

> Merging 4 nodes into parent node.
> Parent node id: 90b3476a-489a-4fee-beb7-1c5363a50ab7.
> Parent node text: [16]
⼈类解决问题的模式通常是⽤最快捷、直观的判断，⽽不是有意识的、⼀步⼀步的推导，早期⼈⼯智能研究通常使⽤逐步推导的⽅式。[17]⼈⼯智能研究已经
于这种“次表征性的”解决问题⽅法获取进展...

> Merging 2 nodes into parent node.
> Parent node id: 7bab41fa-272e-4847-a568-e22dbc7e9fed.
> Parent node text: 依⽬前的研究⽅向，电脑⽆法突变、苏醒、产⽣⾃我意志，AI也不可能具有创意与智能、同情⼼与审美等这⽅⾯的能⼒。
AI逐渐普及后，将会在企业管理中扮演很重要的⾓⾊，⽽⼈类的管理者应 如何适度的调整⾃...

> Merging 5 nodes into parent node.
> Parent node id: b6df9f33-521b-434b-b26d-8f6876a93184.
> Parent node text: 其它关于 动物 或其它⼈造系统的智能也普遍被认为是⼈⼯智能相关的研究课题。
⼈⼯智能⽬前在 电脑 领域内，得到了愈加⼴泛的发挥。并在 机器⼈ 、经济政治决策、 控制系统 、仿真系统 中得到应⽤。...

> Merging 3 nodes into parent node.
> Parent node id: a7a0bda7-af15-46f6-a27f-28cf2b2357e0.
> Parent node text: 2017年6⽉份马云在美国底特律举⾏“链接世界”（Gateway 17）产业⼤会，会上提出⼈⼯智能可能导致第三次世界⼤战，因为前两次产业⾰命都导致两次⼤
战，战争原因并⾮这些创新发明本⾝，⽽是发...

> Merging 1 nodes into parent node.
> Parent node id: 93ed48c1-a43a-4ebe-ab72-46dd57662601.
> Parent node text: 这与认知科学领域中的表征
感知论点是⼀致的:更⾼的智能需要个体的

In [ ]:
# Evaluate the three-layer structure
run_evals(eval_questions_en, tru_recorder_en, auto_merging_engine_1_en)

In [34]:
from trulens_eval import Tru

# Get and display the leaderboard
Tru().get_leaderboard(app_ids=[])

,Context Relevance,Groundedness,Answer Relevance,latency,total_cost
app_id,,,,,
app_0,0.358788,0.888571,0.9400,7.866667,0.0
app_1,NaN,NaN,0.9125,7.866667,0.0


In [35]:
# Run the TruLens dashboard to view detailed evaluation results
Tru().run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.2.7:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

By comparing the TruLens evaluation results of the auto-merge indexes with two- and three-layer structures, we can gain insight into the impact of different hierarchical structures on retrieval performance and optimize our auto-merge strategy accordingly.